<a href="https://colab.research.google.com/github/nonohuang0819/kaggle/blob/main/task2/task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>